# Incrementally saving progress to a file

In many cases it is useful to save the chain to a file. This makes iit easier to post-process a long chain and makes things less disastrous if the computer crashes somewhere in the midle of an expensive MCMC run.

In this recipe we are going to use the callback interface to save the samples and their corresponding log-probability values in a `.h5` file. To do this you need to have [``h5py``](https://docs.h5py.org/en/latest/build.html#pre-built-installation-recommended) installed.

We will set up a simple problem of sampling from a normal/Gaussian distribution as an example:

In [1]:
import zeus
import numpy as np

ndim = 2
nwalkers = 10
nsteps = 1000

def log_prob(x):
    return -0.5*np.dot(x,x)

x0 = 1e-3 * np.random.randn(nwalkers, ndim)

Where ``x0`` is the initial positions of the walkers.

We will then initialise the sampler and start the MCMC run by providing the ``zeus.callbacks.SaveProgressCallback`` callback function.

In [2]:
sampler = zeus.EnsembleSampler(nwalkers, ndim, log_prob)
sampler.run_mcmc(x0, nsteps, callbacks=zeus.callbacks.SaveProgressCallback("saved_chains.h5", ncheck=100))

Initialising ensemble of 10 walkers...
Sampling progress : 100%|██████████| 1000/1000 [00:01<00:00, 656.62it/s]


The above piece of code saved the chain incrementally every ``ncheck=100`` steps to a file named ``saved_chains.h5``. We can now access the chains using the ``h5py`` package as follows:

In [3]:
import h5py 

with h5py.File('saved_chains.h5', "r") as hf:
    samples = np.copy(hf['samples'])
    logprob_samples = np.copy(hf['logprob'])

print(samples.shape)
print(logprob_samples.shape)

(1000, 10, 2)
(1000, 10)
